## 1. mongodb 문법 실습

* cine21 배우 랭킹 사이트 크롤링
  - 사이트 주소: http://www.cine21.com/rank/person/
  - 요청 방식 확인 방법: 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form data
      - section = 'actor'
      - period_start = '2018-08'
      - gender = 'all'
      - page = 3 

#### 배우 랭킹 및 상세 정보 크롤링 및 mongodb 저장 (full source)

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list)

#### mongodb 접속 기본 코드

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('5f17c4855d03ae9996c59546'), '배우이름': '이병헌', '흥행지수': 46630, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '순위': '1', '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름'}
{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}
{'_id': ObjectId('5f17c4855d03ae9996c59548'), '배우이름': '마동석', '흥행지수': 19381, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '순위': '3', '다른 이름': 'Ma Dongseok', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\n

### 1.1. 컬럼명 변경 
* 저장되 있는 mongodb 데이터의 컬럼명을 변경하는 방법
* update_one()/update_many() 함수 활용

In [2]:
actor_collection.update_many ( {}, { '$rename': {'다른 이름':'다른이름'}} )

In [3]:
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('5f17c4855d03ae9996c59546'), '배우이름': '이병헌', '흥행지수': 46630, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '순위': '1', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름', '다른이름': 'Byung-hun Lee;BH Lee'}
{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c59548'), '배우이름': '마동석', '흥행지수': 19381, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '순위': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madong

#### 참고: collection 변수명 변경

In [ ]:
actor = actor_collection

### 1.2 find의 다양한 문법 - sort

In [7]:
docs = actor_collection.find({}).sort('생년월일', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('5f17c4855d03ae9996c595ab'), '배우이름': '허율', '흥행지수': 2708, '출연영화': ['클로젯'], '순위': '102', '직업': '배우', '생년월일': '2009-06-26', '성별': '여'}
{'_id': ObjectId('5f17c4855d03ae9996c595cb'), '배우이름': '김강훈', '흥행지수': 1838, '출연영화': ['엑시트', '변신'], '순위': '134', '직업': '배우', '생년월일': '2009-06-07', '성별': '남', '홈페이지': '\nhttp://instagram.com/kanghoon.juwon\n'}
{'_id': ObjectId('5f17c4855d03ae9996c595ce'), '배우이름': '정지훈', '흥행지수': 1758, '출연영화': ['사자', '덕구', '신과 함께-인과 연'], '순위': '137', '직업': '배우', '생년월일': '2007-05-31', '성별': '남'}
{'_id': ObjectId('5f17c4855d03ae9996c5959f'), '배우이름': '이지원', '흥행지수': 3255, '출연영화': ['히트맨', '개를 훔치는 완벽한 방법'], '순위': '90', '직업': '배우', '생년월일': '2006-08-30', '성별': '여'}
{'_id': ObjectId('5f17c4855d03ae9996c5958f'), '배우이름': '이레', '흥행지수': 4298, '출연영화': ['반도', '너의 이름은.', '증인', '개를 훔치는 완벽한 방법'], '순위': '74', '직업': '배우', '생년월일': '2006-03-12', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/leere.vv/\n'}
{'_id': ObjectId('5f17c4855d03ae9996c59589'), '배우이름': '이재인', '흥행지수': 4464, '출연

In [6]:
docs = actor_collection.find({"생년월일" : {'$exists' : True }}).sort("생년월일").limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c595a9'), '배우이름': '신구', '흥행지수': 2882, '출연영화': ['천문: 하늘에 묻는다', '8월의 크리스마스', '비밥바룰라', '해빙'], '순위': '100', '직업': '배우', '생년월일': '1936-08-13', '성별': '남', '학교': '성균관대학교 국문학 (중퇴)', '취미': '골프, 조깅', '다른이름': '신순기'}
{'_id': ObjectId('5f17c4855d03ae9996c59595'), '배우이름': '나문희', '흥행지수': 3826, '출연영화': ['감쪽같은 그녀', '정직한 후보', '아이 캔 스피크', '수상한 그녀'], '순위': '80', '직업': '배우', '생년월일': '1941-11-30', '성별': '여', '홈페이지': '\nhttp://www.namuni.kr/\n', '학교': '창덕여자고등학교'}
{'_id': ObjectId('5f17c4855d03ae9996c59559'), '배우이름': '박인환', '흥행지수': 10871, '출연영화': ['엑시트', '봄날은 간다', '비밥바룰라', '기묘한 가족', '수상한 그녀'], '순위': '20', '직업': '배우', '생년월일': '1945-01-06', '성별': '남', '학교': '중앙대학교 연극영화과 학사'}
{'_id': ObjectId('5f17c4855d03ae9996c59553'), '배우이름': '고두심', '흥행지수': 11714, '출연영화': ['엑시트'], '순위': '14', '직업': '배우', '생년월일': '1951-05-22', '성별': '여', '홈페이지': '\nhttp://www.kodoosim.com/\n', '신장/체중': '160cm', '학교': '제주여자고등학교'}
{'_id': ObjectId('5f17c4855d03ae9996c595a2'), '배우이름': '안성기', '흥행지수': 3103, '출연영화

### 1.3 find의 다양한 문법 - exists

In [8]:
actor = actor_collection
docs = actor.find({'특기': {'$exists':True}}).sort('흥행지수').limit(5)
for doc in docs:
    print (doc)

{'_id': ObjectId('5f17c4855d03ae9996c595d0'), '배우이름': '이영애', '흥행지수': 1716, '출연영화': ['나를 찾아줘', '봄날은 간다', '친절한 금자씨', '공동경비구역 JSA'], '순위': '139', '직업': '배우', '생년월일': '1971-01-31', '성별': '여', '신장/체중': '165cm, 48kg', '학교': '한양대 독어독문  -중앙대학교대학원 연극영화', '취미': '수영, 승마, 피아노', '특기': '노래', '소속사': '에이스타즈'}
{'_id': ObjectId('5f17c4855d03ae9996c595c3'), '배우이름': '최민호', '흥행지수': 2046, '출연영화': ['장사리 : 잊혀진 영웅들', '인랑'], '순위': '126', '직업': '가수', '생년월일': '1991-12-09', '성별': '남', '신장/체중': '181cm', '학교': '건국대학교 예술학', '취미': '축구, 농구', '특기': '연기, 영어, 중국어', '소속사': 'SM엔터테인먼트', '다른이름': '珉豪; 샤이니; SHINee'}
{'_id': ObjectId('5f17c4855d03ae9996c595bf'), '배우이름': '김태우', '흥행지수': 2096, '출연영화': ['천문: 하늘에 묻는다', '접속', '공동경비구역 JSA', '기담', '창궐'], '순위': '122', '직업': '배우', '생년월일': '1971-04-15', '성별': '남', '신장/체중': '183cm, 73kg', '학교': '중앙대학교 연극영화', '취미': '수영', '특기': '사물놀이'}
{'_id': ObjectId('5f17c4855d03ae9996c595b7'), '배우이름': '김성오', '흥행지수': 2243, '출연영화': ['해치지않아', '불한당: 나쁜 놈들의 세상'], '순위': '114', '직업': '배우', '생년월일': '1978-09-15', 

<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
생년월일이 없는 doc의 actor_name 만 출력하기
</div>

In [9]:
docs = actor.find({'생년월일': {'$exists':False}}, {'배우이름':1, '_id':0})
for doc in docs:
    print (doc)

{'배우이름': '이봉련'}
{'배우이름': '김병순'}
{'배우이름': '배성우'}
{'배우이름': '염정아'}
{'배우이름': '김종수'}
{'배우이름': '김소진'}
{'배우이름': '한정현'}
{'배우이름': '조현정'}
{'배우이름': '김경덕'}
{'배우이름': '신용우'}
{'배우이름': '옥자연'}
{'배우이름': '김영은'}
{'배우이름': '조이현'}
{'배우이름': '장동주'}
{'배우이름': '임지연'}
{'배우이름': '원현준'}


### 1.4 find의 다양한 문법 - 필드값 범위로 검색

In [ ]:
docs = actor.find({'흥행지수': {'$gte': 10000}, '출연영화':'극한직업'}, {'배우이름':1, '출연영화':1, '_id':0}).sort('배우이름').limit(3)
for doc in docs:
    print(doc)

### find의 다양한 문법 - or

In [ ]:
docs = actor.find({'$or': [{'출연영화':'극한직업'}, {'출연영화':'더 킹'}] }, {'배우이름':1, '출연영화':1, '_id':0})
for doc in docs:
    print(doc)

In [ ]:
docs = actor.find({ '흥행지수': {'$gte': 10000}, '$or': [{'출연영화':'극한직업'}, {'출연영화':'더 킹'}] }, {'배우이름':1, '출연영화':1, '_id':0})
for doc in docs:
    print(doc)

### find의 다양한 문법 - nor
* not or

In [ ]:
docs = actor.find({'$nor': [{'흥행지수': { '$gte': 10000}}, {'흥행지수': { '$lte': 2000}}]}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

### find의 다양한 문법 - in, nin
* in: 들어가 있다.
* nin: not in - 들어가 있지 않다.

In [ ]:
docs = actor.find({'흥행지수': { '$in': [9182, 8439]}}, {'배우이름':1, '흥행지수':1, '_id':0})
for doc in docs:
    print (doc)

In [ ]:
docs = actor.find({'흥행지수': { '$nin': [9182, 8439]}}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
흥행지수 가 9182, 8439가 아니고, 10000 이하인 데이터를 3개만 검색하세요. (nor, in, gt 활용, 배우이름과 흥행지수만 출력)
</div>

In [ ]:
docs = actor.find({'$nor': [{'흥행지수': { '$in': [9182, 8439]}}, {'흥행지수': { '$gt': 10000}}]}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

### find의 다양한 문법 - skip, limit
* skip(n): 검색 결과 n개만큼 건너뜀
* limit(n): 검색 결과 n개만 표시

In [ ]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).limit(3)
for doc in docs:
    print (doc)

In [ ]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).skip(3)
for doc in docs:
    print (doc)

In [ ]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).skip(3).limit(3)
for doc in docs:
    print (doc)

### find의 다양한 문법 - list 검색

In [ ]:
docs = actor.find({'출연영화': '극한직업'})
for doc in docs:
    print (doc)

In [ ]:
docs = actor.find({'$or': [{'출연영화': '극한직업'}, {'출연영화': '사바하'}]})
for doc in docs:
    print (doc)

### find의 다양한 문법 - list 검색 (all)

In [ ]:
docs = actor.find({'출연영화': { '$all': ['변산', '사바하']}})
for doc in docs:
    print (doc)

### find의 다양한 문법 - list 검색 (리스트 index 번호로 검색하기)

In [ ]:
docs = actor.find({'출연영화.0': '사바하'})
for doc in docs:
    print (doc)

### find의 다양한 문법 - list 검색 (리스트 사이즈로 검색하기)

In [ ]:
docs = actor.find({'출연영화': {'$size': 5}})
for doc in docs:
    print(doc)

### find의 다양한 문법 (elemMatch)
* 적어도 한 개 이상의 리스트 요소가 복수 개의 조건을 동시에 만족하는 경우

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
elemmatch_sample = actor_db.sample

In [ ]:
elemmatch_sample.insert_many([
    {'results': [82, 85, 88]},
    {'results': [75, 88, 91]}
])

In [ ]:
docs = elemmatch_sample.find({'results': {'$gte': 90, '$lt':85}})
for doc in docs:
    print (doc)

In [ ]:
docs = elemmatch_sample.find({'results': {'$elemMatch': {'$gte':75, '$lt':80}}})
for doc in docs:
    print (doc)

<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
직업이 가수인 배우 중, 흥행지수가 가장 높은 배우순으로 10명을 출력하세요
</div>

In [4]:
docs = actor.find({'직업':'가수'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('5d4541ccc92b652d52161369'), '배우이름': '김설현', '흥행지수': 2944, '출연영화': ['안시성'], '랭킹': '87', '직업': '가수', '생년월일': '1995-01-03', '성별': '여', '홈페이지': '\nhttps://instagram.com/sh_9513\n', '다른이름': '김설현;AOA;에이오에이'}
{'_id': ObjectId('5d4541ccc92b652d52161377'), '배우이름': '도경수', '흥행지수': 2475, '출연영화': ['스윙키즈', '언더독', '신과 함께-죄와 벌', '형', '순정', '7호실'], '랭킹': '101', '직업': '가수', '성별': '남', '다른이름': '엑소;EXO; 디오'}
{'_id': ObjectId('5d4541ccc92b652d52161378'), '배우이름': '진영', '흥행지수': 2469, '출연영화': ['내안의 그놈', '수상한 그녀'], '랭킹': '102', '직업': '가수', '생년월일': '1991-11-18', '성별': '남', '홈페이지': '\nhttps://twitter.com/_jinyoung911118\nhttps://instagram.com/jinyoung0423/\n', '신장/체중': '178cm, 59kg', '다른이름': '정진영; 비원에이포; B1A4'}
{'_id': ObjectId('5d4541ccc92b652d52161387'), '배우이름': '임윤아', '흥행지수': 2176, '출연영화': ['엑시트', '공조'], '랭킹': '117', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}


<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
국가부도의 날에 출연한 배우를 흥행지수가 높은 순으로 10명 출력하세요 
</div>

In [5]:
docs = actor.find({'출연영화':'국가부도의 날'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('5d4541ccc92b652d52161325'), '배우이름': '조한철', '흥행지수': 11833, '출연영화': ['신과 함께-인과 연', '국가부도의 날', '로망', '배심원들'], '랭킹': '19', '직업': '배우', '생년월일': '1973-05-13', '성별': '남'}
{'_id': ObjectId('5d4541ccc92b652d5216132a'), '배우이름': '김홍파', '흥행지수': 10516, '출연영화': ['공작', '국가부도의 날', '마약왕', '말모이', '배심원들', '더 테러 라이브'], '랭킹': '24', '직업': '배우', '생년월일': '1962-01-05', '성별': '남'}
{'_id': ObjectId('5d4541ccc92b652d5216132b'), '배우이름': '조우진', '흥행지수': 10356, '출연영화': ['돈', '창궐', '마약왕', '국가부도의 날', '어쩌다, 결혼', '강철비'], '랭킹': '25', '직업': '배우', '생년월일': '1979-01-16', '성별': '남', '다른이름': '조신제'}
{'_id': ObjectId('5d4541ccc92b652d5216133f'), '배우이름': '염혜란', '흥행지수': 5513, '출연영화': ['국가부도의 날', '미성년', '증인', '걸캅스', '아이 캔 스피크'], '랭킹': '45', '직업': '배우', '성별': '여'}
{'_id': ObjectId('5d4541ccc92b652d52161344'), '배우이름': '송영창', '흥행지수': 5183, '출연영화': ['마약왕', '국가부도의 날', '악질경찰', '말모이', '박쥐', '좋은 놈, 나쁜 놈, 이상한 놈'], '랭킹': '50', '직업': '배우', '생년월일': '1958-04-02', '성별': '남', '학교': '중앙대학교대학원 연극영화학', '취미': '볼링, 비디오감상', '다른이름': '宋永